To train multiple machine learning models, evaluate their performance using metrics (accuracy, precision, recall, F1-score), and perform hyperparameter tuning (using GridSearchCV and RandomizedSearchCV), here's a complete pipeline in Python using scikit-learn.

✅ Step-by-step Implementation:

🔧 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier


📊 2. Load and Prepare Dataset

In [2]:
# Example: Use the breast cancer dataset from sklearn
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


⚙️ 3. Define Models and Hyperparameters


In [3]:
models = {
    "Logistic Regression": {
        "model": LogisticRegression(max_iter=10000),
        "params": {
            "model__C": [0.1, 1, 10],
            "model__penalty": ["l2"],
            "model__solver": ["lbfgs"]
        }
    },
    "Random Forest": {
        "model": RandomForestClassifier(),
        "params": {
            "model__n_estimators": [50, 100, 200],
            "model__max_depth": [None, 10, 20]
        }
    },
    "SVC": {
        "model": SVC(),
        "params": {
            "model__C": [0.1, 1, 10],
            "model__kernel": ["linear", "rbf"]
        }
    },
    "XGBoost": {
        "model": XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
        "params": {
            "model__n_estimators": [50, 100],
            "model__learning_rate": [0.01, 0.1],
            "model__max_depth": [3, 5]
        }
    }
}


🔍 4. Train Models with GridSearchCV and Evaluate

In [4]:
results = []

for model_name, mp in models.items():
    print(f"\nTraining {model_name}...")

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('model', mp['model'])
    ])

    grid = GridSearchCV(pipe, mp['params'], cv=5, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Best Parameters: {grid.best_params_}")
    print(f"Accuracy: {accuracy:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1-score: {f1:.4f}")

    results.append({
        "Model": model_name,
        "Best Params": grid.best_params_,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    })



Training Logistic Regression...
Best Parameters: {'model__C': 1, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
Accuracy: 0.9737 | Precision: 0.9722 | Recall: 0.9859 | F1-score: 0.9790

Training Random Forest...
Best Parameters: {'model__max_depth': 20, 'model__n_estimators': 100}
Accuracy: 0.9561 | Precision: 0.9583 | Recall: 0.9718 | F1-score: 0.9650

Training SVC...
Best Parameters: {'model__C': 0.1, 'model__kernel': 'linear'}
Accuracy: 0.9825 | Precision: 0.9726 | Recall: 1.0000 | F1-score: 0.9861

Training XGBoost...


c:\Users\2004a\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:25:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 50}
Accuracy: 0.9561 | Precision: 0.9583 | Recall: 0.9718 | F1-score: 0.9650


📈 5. Compare Results

In [5]:
results_df = pd.DataFrame(results)
print("\nModel Performance Summary:")
print(results_df.sort_values(by="F1 Score", ascending=False))



Model Performance Summary:
                 Model                                        Best Params  \
2                  SVC       {'model__C': 0.1, 'model__kernel': 'linear'}   
0  Logistic Regression  {'model__C': 1, 'model__penalty': 'l2', 'model...   
1        Random Forest  {'model__max_depth': 20, 'model__n_estimators'...   
3              XGBoost  {'model__learning_rate': 0.1, 'model__max_dept...   

   Accuracy  Precision    Recall  F1 Score  
2  0.982456   0.972603  1.000000  0.986111  
0  0.973684   0.972222  0.985915  0.979021  
1  0.956140   0.958333  0.971831  0.965035  
3  0.956140   0.958333  0.971831  0.965035  


🔁 Use RandomizedSearchCV Instead

In [6]:
random_search = RandomizedSearchCV(pipe, mp['params'], n_iter=10, cv=5, random_state=42, n_jobs=-1)


🧠 Observations

1. ✅ Support Vector Classifier (SVC) achieved the highest F1 Score (0.9861) and perfect Recall (1.0), making it the best model for this classification task.

2. ✅ Logistic Regression performed almost as well with high recall and precision, and is a simpler, more interpretable model.

3. ⚠️ Random Forest and XGBoost showed similar performance, slightly lower than SVC and Logistic Regression. These models might still be preferred in more complex or larger datasets.

📌 Final Recommendation

1. Best Model: SVC with parameters C = 0.1, kernel = 'linear'
This model should be selected for deployment based on current metrics, especially if Recall is critical (e.g., medical diagnosis).

2. Alternative Consideration: If computational efficiency and interpretability are more important, Logistic Regression offers a strong trade-off.